# 🎯 Interactive Machine Learning Workbench
## Build Predictive Models Step-by-Step

This notebook guides you through building machine learning models using a **point-and-click interface**.

**How to use:**
1. Run each cell once (Shift+Enter or click ▶️)
2. Use the **dropdowns and sliders** to make your choices
3. Click the **buttons** to execute each step
4. Read the theory boxes to understand what you're doing

💡 **To see the Python code:** Double-click any cell or click `Show code`

---

In [1]:
#@title ⚙️ **Run This First - Setup** { display-mode: "form" }
#@markdown **Click ▶️ to load all libraries. Wait for ✅ before proceeding.**

# ============================================================
# IMPORTS AND SETUP
# ============================================================
from IPython.display import HTML, display, clear_output
import ipywidgets as widgets
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import io

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression, Ridge, Lasso
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.metrics import (accuracy_score, precision_score, recall_score, f1_score,
                             roc_auc_score, confusion_matrix, roc_curve,
                             mean_squared_error, mean_absolute_error, r2_score)

import warnings
warnings.filterwarnings('ignore')

# ============================================================
# GLOBAL STATE
# ============================================================
class MLState:
    def __init__(self):
        self.reset()

    def reset(self):
        self.df_original = None
        self.df = None
        self.dropped_columns = []
        self.target = None
        self.problem_type = None
        self.X_train = None
        self.X_test = None
        self.y_train = None
        self.y_test = None
        self.X_train_scaled = None
        self.X_test_scaled = None
        self.scaler = None
        self.scaled_columns = []
        self.feature_names = []
        self.models = {}
        self.results = {}
        self.analysis_log = []
        # Cost matrix
        self.cost_tp = 15
        self.cost_fp = -5
        self.cost_tn = 0
        self.cost_fn = -10

    def log(self, message):
        timestamp = datetime.now().strftime("%H:%M:%S")
        self.analysis_log.append(f"[{timestamp}] {message}")

    def get_available_columns(self):
        if self.df is None:
            return []
        cols = [c for c in self.df.columns if c not in self.dropped_columns]
        if self.target and self.target in cols:
            cols.remove(self.target)
        return cols

    def get_categorical_columns(self):
        if self.df is None:
            return []
        cat_cols = self.df.select_dtypes(include=['object', 'category']).columns.tolist()
        return [c for c in cat_cols if c not in self.dropped_columns and c != self.target]

    def get_numeric_columns(self):
        if self.df is None:
            return []
        num_cols = self.df.select_dtypes(include=[np.number]).columns.tolist()
        return [c for c in num_cols if c not in self.dropped_columns and c != self.target]

state = MLState()

# ============================================================
# HELPER: INFO BOX
# ============================================================
def info_box(text, box_type="theory"):
    colors = {
        "info": ("#e7f3fe", "#2196F3"),
        "warning": ("#fff3cd", "#ff9800"),
        "success": ("#d4edda", "#28a745"),
        "theory": ("#f0f4f8", "#5c6bc0")
    }
    bg, border = colors.get(box_type, colors["theory"])
    return widgets.HTML(
        value=f'<div style="background-color:{bg}; border-left:4px solid {border}; padding:12px; margin:10px 0; border-radius:4px;">{text}</div>'
    )

print("✅ Setup complete! All libraries loaded.")
print("\n📚 Proceed to Step 1 below.")

✅ Setup complete! All libraries loaded.

📚 Proceed to Step 1 below.


---
# 📊 STEP 1: Load Your Data

In [2]:
# Direct code to load your local file in VS Code
state.reset()
state.df_original = pd.read_csv('ShopNow Dataset.csv')
state.df = state.df_original.copy()
state.log("Loaded local data: ShopNow Dataset.csv")

# Print the confirmation details as the original cell does
print(f"✅ DATA LOADED SUCCESSFULLY!")
print(f"📊 Shape: {state.df.shape[0]:,} rows × {state.df.shape[1]} columns")
display(state.df.head())

✅ DATA LOADED SUCCESSFULLY!
📊 Shape: 6,000 rows × 19 columns


,customer_id,age,gender,income_bucket,orders_last_12m,recency_days,avg_order_value,website_visits_30d,email_opens_90d,top_category,cat_share_electronics,cat_share_apparel,cat_share_home,cat_share_beauty,cat_share_groceries,cat_share_sports,discount_exposed_30d,feedback_text,purchase_next_30d
0,1,40,M,Mid,4,185,80.41,6,2,Groceries,0.031,0.148,0.059,0.253,0.347,0.162,0,"Customer service was unhelpful, slow to respon...",0
1,2,31,M,High,5,146,171.52,6,5,Home,0.273,0.090,0.476,0.031,0.050,0.080,0,"Customer service was unhelpful, slow to respon...",0
2,3,43,F,Low,1,546,22.90,1,7,Groceries,0.071,0.008,0.155,0.275,0.434,0.057,0,"Customer service was unhelpful, slow to respon...",0
3,4,24,M,Low,8,37,38.67,7,4,Home,0.208,0.125,0.272,0.132,0.082,0.181,0,"The quality of the product felt poor, material...",0
4,5,29,F,Low,15,71,49.69,13,5,Sports,0.073,0.084,0.205,0.020,0.293,0.324,0,The price felt fair and reasonable compared to...,0


---
# 🎯 STEP 2: Select Target Variable & Problem Type

In [3]:
#@title 🎯 **Step 2: Set Target & Problem Type** { display-mode: "form" }
#@markdown **Run this cell, select your target variable, then click Confirm.**

theory = info_box("""
<b>💡 Classification vs Regression:</b><br>
<ul>
<li><b>Classification</b>: Target is a <b>category</b> (yes/no, spam/not spam)<br>
   → We predict <i>which group</i> something belongs to</li>
<li><b>Regression</b>: Target is a <b>continuous number</b> (price, sales)<br>
   → We predict <i>how much</i> or <i>what value</i></li>
</ul>
<b>Example:</b> Predicting if a customer will buy (yes/no) = Classification.<br>
Predicting how much they'll spend = Regression.
""")

# Dropdown for target (will be populated dynamically)
target_dropdown = widgets.Dropdown(
    options=['-- Load data first --'],
    description='Target:',
    style={'description_width': '80px'},
    layout=widgets.Layout(width='300px')
)

problem_dropdown = widgets.Dropdown(
    options=['Classification', 'Regression'],
    value='Classification',
    description='Type:',
    style={'description_width': '80px'},
    layout=widgets.Layout(width='200px')
)

refresh_btn = widgets.Button(description='🔄 Refresh Options', button_style='info', layout=widgets.Layout(width='150px'))
confirm_btn = widgets.Button(description='✅ Confirm Selection', button_style='success', layout=widgets.Layout(width='170px'))

target_output = widgets.Output()

def refresh_target(b):
    if state.df is not None:
        target_dropdown.options = list(state.df.columns)
        with target_output:
            clear_output()
            print("✅ Options refreshed!")

def confirm_target(b):
    with target_output:
        clear_output()
        if state.df is None:
            print("❌ Please load data first (Step 1)!")
            return

        state.target = target_dropdown.value
        state.problem_type = problem_dropdown.value.lower()
        state.log(f"Target: {state.target} ({state.problem_type})")

        print("="*60)
        print("✅ TARGET CONFIGURED")
        print("="*60)
        print(f"\n🎯 Target Variable: {state.target}")
        print(f"📋 Problem Type: {state.problem_type.upper()}")

        if state.problem_type == 'classification':
            print(f"\n📊 Class Distribution:")
            dist = state.df[state.target].value_counts()
            for cls, count in dist.items():
                pct = count / len(state.df) * 100
                print(f"   {cls}: {count:,} ({pct:.1f}%)")
            if len(dist) == 2:
                ratio = dist.max() / dist.min()
                if ratio > 3:
                    print(f"\n⚠️  Class Imbalance! Ratio: {ratio:.1f}:1")
        else:
            stats = state.df[state.target].describe()
            print(f"\n📊 Target Statistics:")
            print(f"   Mean: {stats['mean']:.2f}, Std: {stats['std']:.2f}")
            print(f"   Min: {stats['min']:.2f}, Max: {stats['max']:.2f}")

refresh_btn.on_click(refresh_target)
confirm_btn.on_click(confirm_target)

display(widgets.VBox([
    theory,
    refresh_btn,
    widgets.HBox([target_dropdown, problem_dropdown]),
    confirm_btn,
    target_output
]))

---
# 🗑️ STEP 3: Drop Unnecessary Columns

In [4]:
#@title 🗑️ **Step 3: Drop Columns** { display-mode: "form" }
#@markdown **Run this cell, select columns to drop, then click Drop.**

theory = info_box("""
<b>💡 What columns should you drop?</b><br>
<ul>
<li><b>Identifiers</b>: customer_id, transaction_id → unique per row, no predictive value</li>
<li><b>Free Text</b>: comments, descriptions → requires special NLP processing</li>
<li><b>Redundant</b>: If two columns contain the same information</li>
<li><b>Data Leakage</b>: Variables not available at prediction time</li>
</ul>
<b>⚠️ Once dropped, columns won't appear in later steps!</b>
""")

drop_select = widgets.SelectMultiple(
    options=['-- Load data first --'],
    description='Select:',
    layout=widgets.Layout(width='400px', height='150px'),
    style={'description_width': '60px'}
)

refresh_drop_btn = widgets.Button(description='🔄 Refresh List', button_style='info', layout=widgets.Layout(width='130px'))
drop_btn = widgets.Button(description='🗑️ Drop Selected', button_style='warning', layout=widgets.Layout(width='150px'))

drop_output = widgets.Output()

def refresh_drop(b):
    if state.df is not None:
        drop_select.options = state.get_available_columns()
        with drop_output:
            clear_output()
            print(f"✅ {len(drop_select.options)} columns available")

def drop_columns(b):
    with drop_output:
        clear_output()
        if len(drop_select.value) == 0:
            print("ℹ️ No columns selected")
            return

        for col in drop_select.value:
            if col in state.df.columns:
                state.df = state.df.drop(columns=[col])
                state.dropped_columns.append(col)

        state.log(f"Dropped: {list(drop_select.value)}")

        print("✅ Columns dropped!")
        print(f"\n🗑️ Dropped: {list(drop_select.value)}")
        print(f"\n📋 Remaining ({len(state.get_available_columns())}):")
        for col in state.get_available_columns():
            print(f"   • {col}")

        # Refresh the list
        drop_select.options = state.get_available_columns()

refresh_drop_btn.on_click(refresh_drop)
drop_btn.on_click(drop_columns)

display(widgets.VBox([
    theory,
    widgets.HTML("<i>Hold Ctrl/Cmd to select multiple columns:</i>"),
    refresh_drop_btn,
    drop_select,
    drop_btn,
    drop_output
]))

---
# 🔧 STEP 4: Handle Missing Values

In [ ]:
#@title 🔧 **Step 4: Handle Missing Values** { display-mode: "form" }
#@markdown **Run this cell, select strategies, then click Apply.**

theory = info_box("""
<b>💡 Missing Value Strategies:</b><br><br>
<b>For Numeric Variables:</b>
<ul>
<li><b>Mean</b>: Replace with average → Good for normal distributions</li>
<li><b>Median</b>: Replace with middle value → Better for skewed data</li>
<li><b>Zero</b>: Replace with 0 → Only if 0 is meaningful</li>
</ul>
<b>For Categorical Variables:</b>
<ul>
<li><b>Mode</b>: Replace with most frequent category</li>
<li><b>Unknown</b>: Create a new "Unknown" category</li>
</ul>
""")

numeric_strategy = widgets.Dropdown(
    options=[('Median (recommended)', 'median'), ('Mean', 'mean'), ('Zero', 'zero'), ('Drop Rows', 'drop')],
    value='median',
    description='Numeric:',
    style={'description_width': '80px'},
    layout=widgets.Layout(width='250px')
)

categorical_strategy = widgets.Dropdown(
    options=[('Mode (most frequent)', 'mode'), ('Mark as "Unknown"', 'unknown'), ('Drop Rows', 'drop')],
    value='mode',
    description='Categorical:',
    style={'description_width': '80px'},
    layout=widgets.Layout(width='250px')
)

check_missing_btn = widgets.Button(description='🔍 Check Missing', button_style='info', layout=widgets.Layout(width='140px'))
apply_missing_btn = widgets.Button(description='✨ Apply Strategy', button_style='success', layout=widgets.Layout(width='150px'))

missing_output = widgets.Output()

def check_missing(b):
    with missing_output:
        clear_output()
        if state.df is None:
            print("❌ Load data first!")
            return
        missing = state.df.isnull().sum()
        missing = missing[missing > 0]
        if len(missing) > 0:
            print("⚠️ MISSING VALUES:")
            for col, count in missing.items():
                pct = count / len(state.df) * 100
                print(f"   {col}: {count:,} ({pct:.1f}%)")
        else:
            print("✅ No missing values!")

def apply_missing(b):
    with missing_output:
        clear_output()
        if state.df is None:
            print("❌ Load data first!")
            return

        num_cols = state.get_numeric_columns()
        cat_cols = state.get_categorical_columns()

        # Numeric
        strat = numeric_strategy.value
        if strat == 'mean':
            for col in num_cols:
                state.df[col] = state.df[col].fillna(state.df[col].mean())
        elif strat == 'median':
            for col in num_cols:
                state.df[col] = state.df[col].fillna(state.df[col].median())
        elif strat == 'zero':
            for col in num_cols:
                state.df[col] = state.df[col].fillna(0)
        elif strat == 'drop':
            state.df = state.df.dropna(subset=num_cols)

        # Categorical
        strat = categorical_strategy.value
        if strat == 'mode':
            for col in cat_cols:
                mode = state.df[col].mode()
                state.df[col] = state.df[col].fillna(mode[0] if len(mode) > 0 else 'Unknown')
        elif strat == 'unknown':
            for col in cat_cols:
                state.df[col] = state.df[col].fillna('Unknown')
        elif strat == 'drop':
            state.df = state.df.dropna(subset=cat_cols)

        state.log(f"Missing values handled")
        print("✅ Missing values handled!")
        print(f"\n📏 Dataset: {state.df.shape[0]:,} rows × {state.df.shape[1]} columns")
        remaining = state.df.isnull().sum().sum()
        print(f"{'✅ No' if remaining == 0 else '⚠️ ' + str(remaining)} missing values remain")

check_missing_btn.on_click(check_missing)
apply_missing_btn.on_click(apply_missing)

display(widgets.VBox([
    theory,
    widgets.HBox([numeric_strategy, categorical_strategy]),
    widgets.HBox([check_missing_btn, apply_missing_btn]),
    missing_output
]))

---
# 📊 STEP 5: Handle Outliers

In [ ]:
#@title 📊 **Step 5: Handle Outliers** { display-mode: "form" }
#@markdown **Run this cell, select strategy, then click Apply.**

theory = info_box("""
<b>💡 Outlier Handling Strategies:</b><br>
<ul>
<li><b>None</b>: Keep all data → Use if outliers are meaningful (e.g., fraud)</li>
<li><b>Clip (IQR)</b>: Cap values at 1.5×IQR → Standard statistical method</li>
<li><b>Clip (Percentile)</b>: Cap at specific percentiles → More control</li>
<li><b>Remove</b>: Delete outlier rows → Use cautiously, loses data</li>
</ul>
<b>⚠️ Tree models (RF, XGBoost) are robust to outliers. Linear models are sensitive.</b>
""")

outlier_dropdown = widgets.Dropdown(
    options=[
        ('None - Keep all data', 'none'),
        ('Clip using IQR (1.5×)', 'clip_iqr'),
        ('Clip at percentiles', 'clip_pct'),
        ('Remove outlier rows', 'remove')
    ],
    value='none',
    description='Strategy:',
    style={'description_width': '70px'},
    layout=widgets.Layout(width='280px')
)

lower_pct = widgets.IntSlider(value=1, min=0, max=10, description='Lower %:', style={'description_width': '70px'})
upper_pct = widgets.IntSlider(value=99, min=90, max=100, description='Upper %:', style={'description_width': '70px'})

apply_outlier_btn = widgets.Button(description='🔧 Apply', button_style='success', layout=widgets.Layout(width='120px'))
outlier_output = widgets.Output()

def apply_outliers(b):
    with outlier_output:
        clear_output()
        if state.df is None:
            print("❌ Load data first!")
            return

        num_cols = state.get_numeric_columns()
        original_len = len(state.df)
        strategy = outlier_dropdown.value

        if strategy == 'none':
            print("ℹ️ No outlier handling applied")
        elif strategy == 'clip_iqr':
            for col in num_cols:
                Q1, Q3 = state.df[col].quantile([0.25, 0.75])
                IQR = Q3 - Q1
                state.df[col] = state.df[col].clip(Q1 - 1.5*IQR, Q3 + 1.5*IQR)
            print("✅ Outliers clipped using IQR")
        elif strategy == 'clip_pct':
            for col in num_cols:
                lo = state.df[col].quantile(lower_pct.value / 100)
                hi = state.df[col].quantile(upper_pct.value / 100)
                state.df[col] = state.df[col].clip(lo, hi)
            print(f"✅ Clipped at {lower_pct.value}th and {upper_pct.value}th percentiles")
        elif strategy == 'remove':
            for col in num_cols:
                Q1, Q3 = state.df[col].quantile([0.25, 0.75])
                IQR = Q3 - Q1
                mask = (state.df[col] >= Q1 - 1.5*IQR) & (state.df[col] <= Q3 + 1.5*IQR)
                state.df = state.df[mask]
            print(f"✅ Removed {original_len - len(state.df):,} rows")

        state.log(f"Outliers: {strategy}")
        print(f"\n📏 Dataset: {state.df.shape[0]:,} rows")

apply_outlier_btn.on_click(apply_outliers)

display(widgets.VBox([
    theory,
    outlier_dropdown,
    widgets.HTML("<i>Percentile bounds (only for 'Clip at percentiles'):</i>"),
    widgets.HBox([lower_pct, upper_pct]),
    apply_outlier_btn,
    outlier_output
]))

---
# 🔠 STEP 6: Encode Categorical Variables

In [ ]:
#@title 🔠 **Step 6: Encode Categorical Variables** { display-mode: "form" }
#@markdown **Run this cell, select columns and encoding type, then click Apply.**

theory = info_box("""
<b>💡 Encoding Strategies:</b><br><br>
<b>One-Hot Encoding</b> (for unordered categories):
<ul>
<li>Creates binary columns for each category</li>
<li>Example: Color (Red, Blue) → Color_Red, Color_Blue</li>
<li>Use for: gender, country, product type</li>
</ul>
<b>Ordinal Encoding</b> (for ordered categories):
<ul>
<li>Converts to numbers: Low=0, Medium=1, High=2</li>
<li>Use for: education level, satisfaction rating</li>
<li>⚠️ You must specify the correct order!</li>
</ul>
""")

onehot_select = widgets.SelectMultiple(
    options=['-- Refresh to see columns --'],
    description='One-Hot:',
    layout=widgets.Layout(width='250px', height='120px'),
    style={'description_width': '70px'}
)

ordinal_dropdown = widgets.Dropdown(
    options=['-- Select --'],
    description='Ordinal:',
    style={'description_width': '70px'},
    layout=widgets.Layout(width='250px')
)

ordinal_order_text = widgets.Text(
    placeholder='e.g., Low, Medium, High (low to high)',
    description='Order:',
    style={'description_width': '70px'},
    layout=widgets.Layout(width='350px')
)

refresh_encode_btn = widgets.Button(description='🔄 Refresh', button_style='info', layout=widgets.Layout(width='100px'))
apply_onehot_btn = widgets.Button(description='🔠 Apply One-Hot', button_style='success', layout=widgets.Layout(width='150px'))
apply_ordinal_btn = widgets.Button(description='🔢 Apply Ordinal', button_style='success', layout=widgets.Layout(width='150px'))

encode_output = widgets.Output()

def refresh_encode(b):
    if state.df is not None:
        cat_cols = state.get_categorical_columns()
        onehot_select.options = cat_cols if cat_cols else ['-- No categorical columns --']
        ordinal_dropdown.options = ['-- Select --'] + cat_cols
        with encode_output:
            clear_output()
            if cat_cols:
                print(f"📋 Categorical columns: {cat_cols}")
                for col in cat_cols:
                    unique = list(state.df[col].unique())[:5]
                    print(f"   {col}: {unique}{'...' if len(state.df[col].unique()) > 5 else ''}")

def apply_onehot(b):
    with encode_output:
        clear_output()
        if len(onehot_select.value) == 0:
            print("ℹ️ No columns selected")
            return
        cols = [c for c in onehot_select.value if c in state.df.columns]
        if cols:
            state.df = pd.get_dummies(state.df, columns=cols, drop_first=True)
            state.log(f"One-hot encoded: {cols}")
            print(f"✅ One-hot encoded: {cols}")
            print(f"\n📏 Dataset: {state.df.shape[1]} columns now")
            # Refresh the list
            refresh_encode(None)

def apply_ordinal(b):
    with encode_output:
        clear_output()
        col = ordinal_dropdown.value
        if col == '-- Select --' or not ordinal_order_text.value:
            print("ℹ️ Select column and enter order")
            return
        order = [x.strip() for x in ordinal_order_text.value.split(',')]
        order_map = {val: idx for idx, val in enumerate(order)}
        state.df[col] = state.df[col].map(order_map)
        state.log(f"Ordinal encoded: {col}")
        print(f"✅ Ordinal encoded '{col}': {order_map}")
        # Refresh the list
        refresh_encode(None)

refresh_encode_btn.on_click(refresh_encode)
apply_onehot_btn.on_click(apply_onehot)
apply_ordinal_btn.on_click(apply_ordinal)

display(widgets.VBox([
    theory,
    refresh_encode_btn,
    widgets.HTML("<b>One-Hot Encoding:</b> <i>(Hold Ctrl/Cmd for multiple)</i>"),
    onehot_select,
    apply_onehot_btn,
    widgets.HTML("<br><b>Ordinal Encoding:</b> <i>(one at a time)</i>"),
    widgets.HBox([ordinal_dropdown, ordinal_order_text]),
    apply_ordinal_btn,
    encode_output
]))

---
# ⚖️ STEP 7: Scale Features & Split Data

In [8]:
#@title ⚖️ **Step 7: Scale & Split Data** { display-mode: "form" }
#@markdown **Run this cell, select columns to scale, set test size, then click Split.**

theory = info_box("""
<b>💡 Why Scale?</b>
<ul>
<li><b>Linear models (Ridge, Lasso)</b>: REQUIRE scaling - variables with larger ranges dominate</li>
<li><b>Tree models (RF, XGBoost)</b>: DON'T need scaling - they split on values</li>
</ul>
<b>💡 Why Split?</b>
<ul>
<li><b>Training Set</b>: Model learns from this data</li>
<li><b>Test Set</b>: Held out to evaluate generalization</li>
</ul>
<b>⚠️ Always fit scaler on TRAINING data only, then transform both train and test!</b>
""")

scale_select = widgets.SelectMultiple(
    options=['-- Refresh to see columns --'],
    description='Scale:',
    layout=widgets.Layout(width='300px', height='120px'),
    style={'description_width': '50px'}
)

test_size_slider = widgets.FloatSlider(
    value=0.2, min=0.1, max=0.4, step=0.05,
    description='Test Size:',
    readout_format='.0%',
    style={'description_width': '80px'},
    layout=widgets.Layout(width='350px')
)

random_seed_input = widgets.IntText(value=42, description='Seed:', style={'description_width': '50px'}, layout=widgets.Layout(width='120px'))

refresh_scale_btn = widgets.Button(description='🔄 Refresh', button_style='info', layout=widgets.Layout(width='100px'))
split_btn = widgets.Button(description='✂️ Split Data', button_style='success', layout=widgets.Layout(width='150px'))

split_output = widgets.Output()

def refresh_scale(b):
    if state.df is not None:
        scale_select.options = state.get_numeric_columns()
        with split_output:
            clear_output()
            print(f"📊 {len(scale_select.options)} numeric columns available")

def split_data(b):
    with split_output:
        clear_output()
        if state.df is None or state.target is None:
            print("❌ Complete Steps 1-2 first!")
            return

        X = state.df.drop(columns=[state.target])
        y = state.df[state.target]
        state.feature_names = list(X.columns)

        if state.problem_type == 'classification':
            X_train, X_test, y_train, y_test = train_test_split(
                X, y, test_size=test_size_slider.value, random_state=random_seed_input.value, stratify=y
            )
        else:
            X_train, X_test, y_train, y_test = train_test_split(
                X, y, test_size=test_size_slider.value, random_state=random_seed_input.value
            )

        state.X_train, state.X_test = X_train, X_test
        state.y_train, state.y_test = y_train, y_test

        # Scaling
        state.scaled_columns = [c for c in scale_select.value if c in X_train.columns]
        state.X_train_scaled = X_train.copy()
        state.X_test_scaled = X_test.copy()

        if state.scaled_columns:
            state.scaler = StandardScaler()
            state.X_train_scaled[state.scaled_columns] = state.scaler.fit_transform(X_train[state.scaled_columns])
            state.X_test_scaled[state.scaled_columns] = state.scaler.transform(X_test[state.scaled_columns])

        state.log(f"Split: {len(X_train)} train, {len(X_test)} test")

        print("="*60)
        print("✅ DATA SPLIT COMPLETE!")
        print("="*60)
        print(f"\n📊 Training: {len(X_train):,} samples ({(1-test_size_slider.value)*100:.0f}%)")
        print(f"📊 Test: {len(X_test):,} samples ({test_size_slider.value*100:.0f}%)")
        print(f"📐 Features: {len(state.feature_names)}")
        if state.scaled_columns:
            print(f"\n⚖️ Scaled: {state.scaled_columns}")
        print("\n🎉 Ready to train models!")

refresh_scale_btn.on_click(refresh_scale)
split_btn.on_click(split_data)

display(widgets.VBox([
    theory,
    widgets.HTML("<b>Select columns to scale</b> <i>(for Ridge/Lasso)</i>:"),
    refresh_scale_btn,
    scale_select,
    widgets.HBox([test_size_slider, random_seed_input]),
    split_btn,
    split_output
]))

---
# 💰 STEP 8: Define Cost Matrix

In [ ]:
#@title 💰 **Step 8: Define Cost Matrix** { display-mode: "form" }
#@markdown **Run this cell, enter costs/benefits, then click Save.**

theory = info_box("""
<b>💡 Understanding the Cost Matrix:</b><br><br>
<table style="border-collapse:collapse; width:100%;">
<tr><th></th><th style="border:1px solid #ddd; padding:8px;">Predicted Positive</th><th style="border:1px solid #ddd; padding:8px;">Predicted Negative</th></tr>
<tr><td style="border:1px solid #ddd; padding:8px;"><b>Actual Positive</b></td><td style="border:1px solid #ddd; padding:8px; background:#d4edda;">TP ✅ Correct</td><td style="border:1px solid #ddd; padding:8px; background:#f8d7da;">FN ❌ Missed</td></tr>
<tr><td style="border:1px solid #ddd; padding:8px;"><b>Actual Negative</b></td><td style="border:1px solid #ddd; padding:8px; background:#fff3cd;">FP ⚠️ Wasted</td><td style="border:1px solid #ddd; padding:8px; background:#d4edda;">TN ✅ Correct</td></tr>
</table>
<br><b>Example (Marketing):</b> TP=+\$15 (revenue), FP=-\$5 (wasted cost), TN=\$0, FN=-\$10 (missed sale)
""")

cost_tp_input = widgets.FloatText(value=15, description='TP Value ($):', style={'description_width': '100px'}, layout=widgets.Layout(width='180px'))
cost_fp_input = widgets.FloatText(value=-5, description='FP Cost ($):', style={'description_width': '100px'}, layout=widgets.Layout(width='180px'))
cost_tn_input = widgets.FloatText(value=0, description='TN Value ($):', style={'description_width': '100px'}, layout=widgets.Layout(width='180px'))
cost_fn_input = widgets.FloatText(value=-10, description='FN Cost ($):', style={'description_width': '100px'}, layout=widgets.Layout(width='180px'))

save_value_btn = widgets.Button(description='💾 Save Costs', button_style='success', layout=widgets.Layout(width='130px'))
cost_output = widgets.Output()

def save_costs(b):
    with cost_output:
        clear_output()
        state.cost_tp = cost_tp_input.value
        state.cost_fp = cost_fp_input.value
        state.cost_tn = cost_tn_input.value
        state.cost_fn = cost_fn_input.value
        print("✅ Cost matrix saved!")
        print(f"\n   TP: ${state.cost_tp}  |  FN: ${state.cost_fn}")
        print(f"   FP: ${state.cost_fp}  |  TN: ${state.cost_tn}")

save_value_btn.on_click(save_costs)

display(widgets.VBox([
    theory,
    widgets.HTML("<b>Enter positive values for benefits, negative for costs:</b>"),
    widgets.HBox([cost_tp_input, cost_fn_input]),
    widgets.HBox([cost_fp_input, cost_tn_input]),
    save_value_btn,
    cost_output
]))

---
---
# 🤖 MODEL TRAINING
## Train each model and see results immediately
---

In [10]:
#@title 🔵 **Ridge Model** { display-mode: "form" }
#@markdown **Ridge (L2 Regularization):** Shrinks coefficients but keeps all features.

theory = info_box("""
<b>💡 Ridge (L2 Regularization):</b>
<ul>
<li>Adds penalty to large coefficients → prevents overfitting</li>
<li>Shrinks coefficients toward zero but <b>never exactly to zero</b></li>
<li>Best when <b>all features are somewhat relevant</b></li>
<li><b>C</b> = inverse regularization (lower C = stronger penalty)</li>
</ul>
""")

ridge_c_slider = widgets.FloatSlider(value=1.0, min=0.01, max=10, step=0.1, description='C:',
                                      style={'description_width': '30px'}, layout=widgets.Layout(width='350px'))
ridge_importance_check = widgets.Checkbox(value=True, description='Show Feature Importance', style={'description_width': 'initial'})

train_ridge_btn = widgets.Button(description='🚀 Train Ridge', button_style='primary', layout=widgets.Layout(width='150px', height='40px'))
ridge_output = widgets.Output()

def train_ridge(b):
    with ridge_output:
        clear_output()
        if state.X_train is None:
            print("❌ Complete data preparation first (Steps 1-7)!")
            return

        print("🔄 Training Ridge...")

        if state.problem_type == 'classification':
            model = LogisticRegression(penalty='l2', C=ridge_c_slider.value, solver='lbfgs', max_iter=1000, random_state=42)
            model.fit(state.X_train_scaled, state.y_train)
            y_pred = model.predict(state.X_test_scaled)
            y_prob = model.predict_proba(state.X_test_scaled)[:, 1]

            cm = confusion_matrix(state.y_test, y_pred)
            tn, fp, fn, tp = cm.ravel()

            state.models['Ridge'] = model
            state.results['Ridge'] = {
                'y_pred': y_pred, 'y_prob': y_prob,
                'accuracy': accuracy_score(state.y_test, y_pred),
                'precision': precision_score(state.y_test, y_pred, zero_division=0),
                'recall': recall_score(state.y_test, y_pred, zero_division=0),
                'f1': f1_score(state.y_test, y_pred, zero_division=0),
                'auc': roc_auc_score(state.y_test, y_prob),
                'tp': tp, 'fp': fp, 'tn': tn, 'fn': fn
            }

            total_cost = tp*state.cost_tp + fp*state.cost_fp + tn*state.cost_tn + fn*state.cost_fn
            state.results['Ridge']['total_cost'] = total_cost
            state.results['Ridge']['avg_cost'] = total_cost / len(state.y_test)

            print("="*60)
            print("✅ RIDGE TRAINED!")
            print("="*60)
            print(f"\n📊 METRICS:")
            print(f"   Accuracy: {state.results['Ridge']['accuracy']:.4f}  |  AUC: {state.results['Ridge']['auc']:.4f}")
            print(f"   Precision: {state.results['Ridge']['precision']:.4f}  |  Recall: {state.results['Ridge']['recall']:.4f}")
            print(f"\n📋 CONFUSION MATRIX:  TP={tp} | FP={fp} | TN={tn} | FN={fn}")
            print(f"\n💰 VALUE: Total=${total_cost:,.2f}  |  Avg=${total_cost/len(state.y_test):.2f}")

            if ridge_importance_check.value:
                print(f"\n📊 TOP 10 FEATURES:")
                coef = model.coef_.flatten()
                imp_df = pd.DataFrame({'Feature': state.feature_names, 'Coefficient': coef})
                imp_df['|Coef|'] = imp_df['Coefficient'].abs()
                imp_df = imp_df.sort_values('|Coef|', ascending=False).head(10)
                display(imp_df[['Feature', 'Coefficient']].reset_index(drop=True))
        else:
            model = Ridge(alpha=1/ridge_c_slider.value, random_state=42)
            model.fit(state.X_train_scaled, state.y_train)
            y_pred = model.predict(state.X_test_scaled)
            state.models['Ridge'] = model
            state.results['Ridge'] = {
                'y_pred': y_pred,
                'rmse': np.sqrt(mean_squared_error(state.y_test, y_pred)),
                'mae': mean_absolute_error(state.y_test, y_pred),
                'r2': r2_score(state.y_test, y_pred)
            }
            print("="*60)
            print("✅ RIDGE TRAINED!")
            print("="*60)
            print(f"\n📊 RMSE: {state.results['Ridge']['rmse']:.4f}  |  MAE: {state.results['Ridge']['mae']:.4f}  |  R²: {state.results['Ridge']['r2']:.4f}")

        state.log("Ridge trained")

train_ridge_btn.on_click(train_ridge)

display(widgets.VBox([
    theory,
    widgets.HBox([ridge_c_slider, ridge_importance_check]),
    train_ridge_btn,
    ridge_output
]))

In [11]:
#@title 🟢 **Lasso Model** { display-mode: "form" }
#@markdown **Lasso (L1 Regularization):** Can eliminate features by shrinking coefficients to exactly zero.

theory = info_box("""
<b>💡 Lasso (L1 Regularization):</b>
<ul>
<li>Can shrink coefficients <b>exactly to zero</b></li>
<li>Performs <b>automatic feature selection</b></li>
<li>Best when <b>many features are irrelevant</b></li>
<li><b>Ridge vs Lasso:</b> Ridge keeps all features, Lasso eliminates some</li>
</ul>
""")

lasso_c_slider = widgets.FloatSlider(value=1.0, min=0.01, max=10, step=0.1, description='C:',
                                      style={'description_width': '30px'}, layout=widgets.Layout(width='350px'))
lasso_importance_check = widgets.Checkbox(value=True, description='Show Feature Importance', style={'description_width': 'initial'})

train_lasso_btn = widgets.Button(description='🚀 Train Lasso', button_style='primary', layout=widgets.Layout(width='150px', height='40px'))
lasso_output = widgets.Output()

def train_lasso(b):
    with lasso_output:
        clear_output()
        if state.X_train is None:
            print("❌ Complete data preparation first!")
            return

        print("🔄 Training Lasso...")

        if state.problem_type == 'classification':
            model = LogisticRegression(penalty='l1', C=lasso_c_slider.value, solver='saga', max_iter=1000, random_state=42)
            model.fit(state.X_train_scaled, state.y_train)
            y_pred = model.predict(state.X_test_scaled)
            y_prob = model.predict_proba(state.X_test_scaled)[:, 1]

            cm = confusion_matrix(state.y_test, y_pred)
            tn, fp, fn, tp = cm.ravel()
            eliminated = sum(1 for c in model.coef_[0] if c == 0)

            state.models['Lasso'] = model
            state.results['Lasso'] = {
                'y_pred': y_pred, 'y_prob': y_prob,
                'accuracy': accuracy_score(state.y_test, y_pred),
                'precision': precision_score(state.y_test, y_pred, zero_division=0),
                'recall': recall_score(state.y_test, y_pred, zero_division=0),
                'f1': f1_score(state.y_test, y_pred, zero_division=0),
                'auc': roc_auc_score(state.y_test, y_prob),
                'tp': tp, 'fp': fp, 'tn': tn, 'fn': fn
            }

            total_cost = tp*state.cost_tp + fp*state.cost_fp + tn*state.cost_tn + fn*state.cost_fn
            state.results['Lasso']['total_cost'] = total_cost
            state.results['Lasso']['avg_cost'] = total_cost / len(state.y_test)

            print("="*60)
            print("✅ LASSO TRAINED!")
            print("="*60)
            print(f"\n🔍 FEATURE SELECTION: {eliminated}/{len(state.feature_names)} eliminated")
            print(f"\n📊 METRICS:")
            print(f"   Accuracy: {state.results['Lasso']['accuracy']:.4f}  |  AUC: {state.results['Lasso']['auc']:.4f}")
            print(f"   Precision: {state.results['Lasso']['precision']:.4f}  |  Recall: {state.results['Lasso']['recall']:.4f}")
            print(f"\n📋 CONFUSION MATRIX:  TP={tp} | FP={fp} | TN={tn} | FN={fn}")
            print(f"\n💰 VALUE: Total=${total_cost:,.2f}  |  Avg=${total_cost/len(state.y_test):.2f}")

            if lasso_importance_check.value:
                print(f"\n📊 SELECTED FEATURES (non-zero):")
                coef = model.coef_.flatten()
                imp_df = pd.DataFrame({'Feature': state.feature_names, 'Coefficient': coef})
                imp_df = imp_df[imp_df['Coefficient'] != 0]
                imp_df['|Coef|'] = imp_df['Coefficient'].abs()
                imp_df = imp_df.sort_values('|Coef|', ascending=False)
                display(imp_df[['Feature', 'Coefficient']].reset_index(drop=True))
        else:
            model = Lasso(alpha=1/lasso_c_slider.value, max_iter=1000, random_state=42)
            model.fit(state.X_train_scaled, state.y_train)
            y_pred = model.predict(state.X_test_scaled)
            eliminated = sum(1 for c in model.coef_ if c == 0)
            state.models['Lasso'] = model
            state.results['Lasso'] = {
                'y_pred': y_pred,
                'rmse': np.sqrt(mean_squared_error(state.y_test, y_pred)),
                'mae': mean_absolute_error(state.y_test, y_pred),
                'r2': r2_score(state.y_test, y_pred)
            }
            print("="*60)
            print("✅ LASSO TRAINED!")
            print("="*60)
            print(f"\n🔍 Features eliminated: {eliminated}/{len(state.feature_names)}")
            print(f"\n📊 RMSE: {state.results['Lasso']['rmse']:.4f}  |  MAE: {state.results['Lasso']['mae']:.4f}  |  R²: {state.results['Lasso']['r2']:.4f}")

        state.log("Lasso trained")

train_lasso_btn.on_click(train_lasso)

display(widgets.VBox([
    theory,
    widgets.HBox([lasso_c_slider, lasso_importance_check]),
    train_lasso_btn,
    lasso_output
]))

In [12]:
#@title 🌲 **Random Forest Model** { display-mode: "form" }
#@markdown **Random Forest:** Builds many trees and averages predictions. No scaling needed!

theory = info_box("""
<b>💡 Random Forest:</b>
<ul>
<li>Builds <b>many decision trees</b> and averages predictions</li>
<li>Each tree sees <b>random subset</b> of data and features</li>
<li>Handles non-linear relationships, robust to outliers</li>
<li><b>No scaling needed!</b></li>
</ul>
""")

rf_trees_slider = widgets.IntSlider(value=100, min=10, max=500, step=10, description='Trees:', style={'description_width': '50px'}, layout=widgets.Layout(width='300px'))
rf_depth_slider = widgets.IntSlider(value=10, min=2, max=30, step=1, description='Depth:', style={'description_width': '50px'}, layout=widgets.Layout(width='300px'))
rf_importance_check = widgets.Checkbox(value=True, description='Show Feature Importance', style={'description_width': 'initial'})

train_rf_btn = widgets.Button(description='🚀 Train Random Forest', button_style='primary', layout=widgets.Layout(width='180px', height='40px'))
rf_output = widgets.Output()

def train_rf(b):
    with rf_output:
        clear_output()
        if state.X_train is None:
            print("❌ Complete data preparation first!")
            return

        print("🔄 Training Random Forest...")

        if state.problem_type == 'classification':
            model = RandomForestClassifier(n_estimators=rf_trees_slider.value, max_depth=rf_depth_slider.value, random_state=42)
            model.fit(state.X_train, state.y_train)
            y_pred = model.predict(state.X_test)
            y_prob = model.predict_proba(state.X_test)[:, 1]

            cm = confusion_matrix(state.y_test, y_pred)
            tn, fp, fn, tp = cm.ravel()

            state.models['Random Forest'] = model
            state.results['Random Forest'] = {
                'y_pred': y_pred, 'y_prob': y_prob,
                'accuracy': accuracy_score(state.y_test, y_pred),
                'precision': precision_score(state.y_test, y_pred, zero_division=0),
                'recall': recall_score(state.y_test, y_pred, zero_division=0),
                'f1': f1_score(state.y_test, y_pred, zero_division=0),
                'auc': roc_auc_score(state.y_test, y_prob),
                'tp': tp, 'fp': fp, 'tn': tn, 'fn': fn
            }

            total_cost = tp*state.cost_tp + fp*state.cost_fp + tn*state.cost_tn + fn*state.cost_fn
            state.results['Random Forest']['total_cost'] = total_cost
            state.results['Random Forest']['avg_cost'] = total_cost / len(state.y_test)

            print("="*60)
            print("✅ RANDOM FOREST TRAINED!")
            print("="*60)
            print(f"\n🌲 Config: {rf_trees_slider.value} trees, depth {rf_depth_slider.value}")
            print(f"\n📊 METRICS:")
            print(f"   Accuracy: {state.results['Random Forest']['accuracy']:.4f}  |  AUC: {state.results['Random Forest']['auc']:.4f}")
            print(f"   Precision: {state.results['Random Forest']['precision']:.4f}  |  Recall: {state.results['Random Forest']['recall']:.4f}")
            print(f"\n📋 CONFUSION MATRIX:  TP={tp} | FP={fp} | TN={tn} | FN={fn}")
            print(f"\n💰 VALUE: Total=${total_cost:,.2f}  |  Avg=${total_cost/len(state.y_test):.2f}")

            if rf_importance_check.value:
                print(f"\n📊 TOP 10 FEATURES:")
                imp_df = pd.DataFrame({'Feature': state.feature_names, 'Importance': model.feature_importances_})
                imp_df = imp_df.sort_values('Importance', ascending=False).head(10)
                display(imp_df.reset_index(drop=True))
        else:
            model = RandomForestRegressor(n_estimators=rf_trees_slider.value, max_depth=rf_depth_slider.value, random_state=42)
            model.fit(state.X_train, state.y_train)
            y_pred = model.predict(state.X_test)
            state.models['Random Forest'] = model
            state.results['Random Forest'] = {
                'y_pred': y_pred,
                'rmse': np.sqrt(mean_squared_error(state.y_test, y_pred)),
                'mae': mean_absolute_error(state.y_test, y_pred),
                'r2': r2_score(state.y_test, y_pred)
            }
            print("="*60)
            print("✅ RANDOM FOREST TRAINED!")
            print("="*60)
            print(f"\n📊 RMSE: {state.results['Random Forest']['rmse']:.4f}  |  MAE: {state.results['Random Forest']['mae']:.4f}  |  R²: {state.results['Random Forest']['r2']:.4f}")

        state.log("Random Forest trained")

train_rf_btn.on_click(train_rf)

display(widgets.VBox([
    theory,
    widgets.HBox([rf_trees_slider, rf_depth_slider]),
    rf_importance_check,
    train_rf_btn,
    rf_output
]))

In [13]:
#@title 🚀 **XGBoost Model** { display-mode: "form" }
#@markdown **XGBoost (Gradient Boosting):** Builds trees sequentially, each correcting previous errors.

theory = info_box("""
<b>💡 XGBoost (Gradient Boosting):</b>
<ul>
<li>Builds trees <b>sequentially</b>, each correcting previous errors</li>
<li>Often achieves <b>best predictive accuracy</b></li>
<li><b>No scaling needed!</b></li>
<li><b>RF vs XGBoost:</b> RF builds independently (parallel), XGBoost builds sequentially</li>
</ul>
""")

xgb_rounds_slider = widgets.IntSlider(value=100, min=10, max=500, step=10, description='Rounds:', style={'description_width': '60px'}, layout=widgets.Layout(width='280px'))
xgb_depth_slider = widgets.IntSlider(value=6, min=2, max=15, step=1, description='Depth:', style={'description_width': '60px'}, layout=widgets.Layout(width='280px'))
xgb_lr_slider = widgets.FloatSlider(value=0.1, min=0.01, max=0.5, step=0.01, description='Learn Rate:', style={'description_width': '80px'}, layout=widgets.Layout(width='280px'))
xgb_importance_check = widgets.Checkbox(value=True, description='Show Feature Importance', style={'description_width': 'initial'})

train_xgb_btn = widgets.Button(description='🚀 Train XGBoost', button_style='primary', layout=widgets.Layout(width='150px', height='40px'))
xgb_output = widgets.Output()

def train_xgb(b):
    with xgb_output:
        clear_output()
        if state.X_train is None:
            print("❌ Complete data preparation first!")
            return

        print("🔄 Training XGBoost...")

        if state.problem_type == 'classification':
            model = GradientBoostingClassifier(
                n_estimators=xgb_rounds_slider.value, max_depth=xgb_depth_slider.value,
                learning_rate=xgb_lr_slider.value, random_state=42
            )
            model.fit(state.X_train, state.y_train)
            y_pred = model.predict(state.X_test)
            y_prob = model.predict_proba(state.X_test)[:, 1]

            cm = confusion_matrix(state.y_test, y_pred)
            tn, fp, fn, tp = cm.ravel()

            state.models['XGBoost'] = model
            state.results['XGBoost'] = {
                'y_pred': y_pred, 'y_prob': y_prob,
                'accuracy': accuracy_score(state.y_test, y_pred),
                'precision': precision_score(state.y_test, y_pred, zero_division=0),
                'recall': recall_score(state.y_test, y_pred, zero_division=0),
                'f1': f1_score(state.y_test, y_pred, zero_division=0),
                'auc': roc_auc_score(state.y_test, y_prob),
                'tp': tp, 'fp': fp, 'tn': tn, 'fn': fn
            }

            total_cost = tp*state.cost_tp + fp*state.cost_fp + tn*state.cost_tn + fn*state.cost_fn
            state.results['XGBoost']['total_cost'] = total_cost
            state.results['XGBoost']['avg_cost'] = total_cost / len(state.y_test)

            print("="*60)
            print("✅ XGBOOST TRAINED!")
            print("="*60)
            print(f"\n⚡ Config: {xgb_rounds_slider.value} rounds, depth {xgb_depth_slider.value}, LR {xgb_lr_slider.value}")
            print(f"\n📊 METRICS:")
            print(f"   Accuracy: {state.results['XGBoost']['accuracy']:.4f}  |  AUC: {state.results['XGBoost']['auc']:.4f}")
            print(f"   Precision: {state.results['XGBoost']['precision']:.4f}  |  Recall: {state.results['XGBoost']['recall']:.4f}")
            print(f"\n📋 CONFUSION MATRIX:  TP={tp} | FP={fp} | TN={tn} | FN={fn}")
            print(f"\n💰 VALUE: Total=${total_cost:,.2f}  |  Avg=${total_cost/len(state.y_test):.2f}")

            if xgb_importance_check.value:
                print(f"\n📊 TOP 10 FEATURES:")
                imp_df = pd.DataFrame({'Feature': state.feature_names, 'Importance': model.feature_importances_})
                imp_df = imp_df.sort_values('Importance', ascending=False).head(10)
                display(imp_df.reset_index(drop=True))
        else:
            model = GradientBoostingRegressor(
                n_estimators=xgb_rounds_slider.value, max_depth=xgb_depth_slider.value,
                learning_rate=xgb_lr_slider.value, random_state=42
            )
            model.fit(state.X_train, state.y_train)
            y_pred = model.predict(state.X_test)
            state.models['XGBoost'] = model
            state.results['XGBoost'] = {
                'y_pred': y_pred,
                'rmse': np.sqrt(mean_squared_error(state.y_test, y_pred)),
                'mae': mean_absolute_error(state.y_test, y_pred),
                'r2': r2_score(state.y_test, y_pred)
            }
            print("="*60)
            print("✅ XGBOOST TRAINED!")
            print("="*60)
            print(f"\n📊 RMSE: {state.results['XGBoost']['rmse']:.4f}  |  MAE: {state.results['XGBoost']['mae']:.4f}  |  R²: {state.results['XGBoost']['r2']:.4f}")

        state.log("XGBoost trained")

train_xgb_btn.on_click(train_xgb)

display(widgets.VBox([
    theory,
    widgets.HBox([xgb_rounds_slider, xgb_depth_slider]),
    widgets.HBox([xgb_lr_slider, xgb_importance_check]),
    train_xgb_btn,
    xgb_output
]))

---
# 📊 Compare All Models & Calculate Values

In [ ]:
#@title 📊 **Compare Models & Value Analysis** { display-mode: "form" }
#@markdown **Compare all trained models and calculate value on Train or Test set.**

model_dropdown = widgets.Dropdown(
    options=['-- Select Model --'],
    description='Model:',
    style={'description_width': '60px'},
    layout=widgets.Layout(width='200px')
)

dataset_dropdown = widgets.Dropdown(
    options=['Test Set', 'Training Set'],
    description='Dataset:',
    style={'description_width': '60px'},
    layout=widgets.Layout(width='180px')
)

metric_dropdown = widgets.Dropdown(
    options=['Total Value', 'Average Value per Prediction'],
    description='Metric:',
    style={'description_width': '60px'},
    layout=widgets.Layout(width='250px')
)

compare_btn = widgets.Button(description='📊 Compare All Models', button_style='success', layout=widgets.Layout(width='180px'))
value_btn = widgets.Button(description='💰 Calculate Value', button_style='info', layout=widgets.Layout(width='150px'))
refresh_models_btn = widgets.Button(description='🔄 Refresh', button_style='warning', layout=widgets.Layout(width='100px'))

compare_output = widgets.Output()

def refresh_models(b):
    model_dropdown.options = ['-- Select --'] + list(state.models.keys())

def compare_all(b):
    with compare_output:
        clear_output()
        if len(state.results) == 0:
            print("❌ Train at least one model first!")
            return

        print("="*80)
        print("📊 MODEL COMPARISON")
        print("="*80)

        if state.problem_type == 'classification':
            data = [{'Model': name, 'Accuracy': r['accuracy'], 'Precision': r['precision'],
                     'Recall': r['recall'], 'F1': r['f1'], 'AUC': r['auc'],
                     'Total $': r.get('total_cost', 0), 'Avg $': r.get('avg_cost', 0)}
                    for name, r in state.results.items()]
            df = pd.DataFrame(data).round(4)
            display(df)

            best_auc = max(state.results.keys(), key=lambda x: state.results[x]['auc'])
            best_val = max(state.results.keys(), key=lambda x: state.results[x].get('total_cost', -999999))
            print(f"\n🏆 Best AUC: {best_auc} ({state.results[best_auc]['auc']:.4f})")
            print(f"🏆 Best Value: {best_val} (${state.results[best_val].get('total_cost', 0):,.2f})")

            # ROC Curve
            fig, ax = plt.subplots(figsize=(8, 6))
            for name, r in state.results.items():
                fpr, tpr, _ = roc_curve(state.y_test, r['y_prob'])
                ax.plot(fpr, tpr, label=f"{name} ({r['auc']:.3f})")
            ax.plot([0, 1], [0, 1], 'k--')
            ax.set_xlabel('False Positive Rate')
            ax.set_ylabel('True Positive Rate')
            ax.set_title('ROC Curves')
            ax.legend()
            plt.show()
        else:
            data = [{'Model': name, 'RMSE': r['rmse'], 'MAE': r['mae'], 'R²': r['r2']}
                    for name, r in state.results.items()]
            df = pd.DataFrame(data).round(4)
            display(df)
            best_r2 = max(state.results.keys(), key=lambda x: state.results[x]['r2'])
            print(f"\n🏆 Best R²: {best_r2} ({state.results[best_r2]['r2']:.4f})")

def calc_value(b):
    with compare_output:
        clear_output()
        name = model_dropdown.value
        if name == '-- Select --' or name not in state.models:
            print("❌ Select a trained model!")
            return

        model = state.models[name]

        if dataset_dropdown.value == 'Test Set':
            X = state.X_test_scaled if name in ['Ridge', 'Lasso'] else state.X_test
            y_true = state.y_test
        else:
            X = state.X_train_scaled if name in ['Ridge', 'Lasso'] else state.X_train
            y_true = state.y_train

        y_pred = model.predict(X)
        cm = confusion_matrix(y_true, y_pred)
        tn, fp, fn, tp = cm.ravel()

        total = tp*state.cost_tp + fp*state.cost_fp + tn*state.cost_tn + fn*state.cost_fn
        avg = total / len(y_true)

        print("="*60)
        print(f"💰 VALUE ANALYSIS: {name}")
        print(f"📊 Dataset: {dataset_dropdown.value} ({len(y_true):,} obs)")
        print("="*60)
        print(f"\n   TP: {tp:,} × ${state.cost_tp} = ${tp*state.cost_tp:,.2f}")
        print(f"   FP: {fp:,} × ${state.cost_fp} = ${fp*state.cost_fp:,.2f}")
        print(f"   TN: {tn:,} × ${state.cost_tn} = ${tn*state.cost_tn:,.2f}")
        print(f"   FN: {fn:,} × ${state.cost_fn} = ${fn*state.cost_fn:,.2f}")
        print(f"\n" + "="*40)
        if metric_dropdown.value == 'Total Value':
            print(f"💵 TOTAL: ${total:,.2f}")
        else:
            print(f"💵 AVG PER PREDICTION: ${avg:.2f}")

refresh_models_btn.on_click(refresh_models)
compare_btn.on_click(compare_all)
value_btn.on_click(calc_value)

display(widgets.VBox([
    widgets.HBox([compare_btn, refresh_models_btn]),
    widgets.HTML("<br><b>Detailed Value Analysis:</b>"),
    widgets.HBox([model_dropdown, dataset_dropdown, metric_dropdown]),
    value_btn,
    compare_output
]))

In [15]:
#@title 💾 **Save Results** { display-mode: "form" }
#@markdown **Download your model comparison as a CSV file.**

save_btn = widgets.Button(description='💾 Save & Download CSV', button_style='success', layout=widgets.Layout(width='200px', height='40px'))
save_output = widgets.Output()

def save_results(b):
    with save_output:
        clear_output()
        if len(state.results) == 0:
            print("❌ No results to save!")
            return

        if state.problem_type == 'classification':
            data = [{'Model': name, 'Accuracy': r['accuracy'], 'Precision': r['precision'],
                     'Recall': r['recall'], 'F1': r['f1'], 'AUC': r['auc'],
                     'TP': r['tp'], 'FP': r['fp'], 'TN': r['tn'], 'FN': r['fn'],
                     'Total_Value': r.get('total_cost', 0), 'Avg_Value': r.get('avg_cost', 0)}
                    for name, r in state.results.items()]
        else:
            data = [{'Model': name, 'RMSE': r['rmse'], 'MAE': r['mae'], 'R2': r['r2']}
                    for name, r in state.results.items()]

        df = pd.DataFrame(data)
        filename = 'ml_results.csv'
        df.to_csv(filename, index=False)

        print("✅ Results saved!")
        display(df)
        files.download(filename)

save_btn.on_click(save_results)

display(widgets.VBox([save_btn, save_output]))

---
# 🎉 Analysis Complete!

**What you accomplished:**
- ✅ Loaded and explored data
- ✅ Prepared features (missing values, outliers, encoding, scaling)
- ✅ Trained 4 models (Ridge, Lasso, Random Forest, XGBoost)
- ✅ Compared performance metrics
- ✅ Analyzed business value/costs
- ✅ Exported results

**Next steps:** Try different settings and re-train models to see how performance changes!

In [16]:
#@title 📥 **Download Results as HTML** { display-mode: "form" }
#@markdown Run this cell AFTER completing your analysis. It captures all your results.

from IPython.display import HTML, display
from google.colab import files
from datetime import datetime

# Collect all results into HTML
html_content = f"""
<!DOCTYPE html>
<html>
<head>
    <title>ML Analysis Results</title>
    <style>
        body {{ font-family: Arial, sans-serif; margin: 40px; }}
        h1 {{ color: #1a73e8; }}
        h2 {{ color: #5f6368; border-bottom: 1px solid #ddd; padding-bottom: 10px; }}
        table {{ border-collapse: collapse; width: 100%; margin: 20px 0; }}
        th, td {{ border: 1px solid #ddd; padding: 12px; text-align: left; }}
        th {{ background-color: #f8f9fa; }}
        .metric {{ background-color: #e8f5e9; padding: 15px; border-radius: 8px; margin: 10px 0; }}
        .section {{ margin: 30px 0; }}
    </style>
</head>
<body>
    <h1>🎯 Machine Learning Analysis Results</h1>
    <p><em>Generated: {datetime.now().strftime('%Y-%m-%d %H:%M')}</em></p>

    <div class="section">
        <h2>📊 Dataset Summary</h2>
        <ul>
            <li><strong>Target Variable:</strong> {state.target}</li>
            <li><strong>Problem Type:</strong> {state.problem_type}</li>
            <li><strong>Training Samples:</strong> {len(state.X_train) if state.X_train is not None else 'N/A':,}</li>
            <li><strong>Test Samples:</strong> {len(state.X_test) if state.X_test is not None else 'N/A':,}</li>
            <li><strong>Features:</strong> {len(state.feature_names) if state.feature_names else 'N/A'}</li>
        </ul>
    </div>

    <div class="section">
        <h2>💰 Cost Matrix</h2>
        <table>
            <tr><th></th><th>Predicted Positive</th><th>Predicted Negative</th></tr>
            <tr><td><strong>Actual Positive</strong></td><td>TP: ${state.cost_tp}</td><td>FN: ${state.cost_fn}</td></tr>
            <tr><td><strong>Actual Negative</strong></td><td>FP: ${state.cost_fp}</td><td>TN: ${state.cost_tn}</td></tr>
        </table>
    </div>

    <div class="section">
        <h2>🤖 Model Results</h2>
        <table>
            <tr>
                <th>Model</th>
                <th>Accuracy</th>
                <th>Precision</th>
                <th>Recall</th>
                <th>F1</th>
                <th>AUC</th>
                <th>Total Value</th>
            </tr>
"""

# Add model results
if state.problem_type == 'classification':
    for name, r in state.results.items():
        html_content += f"""
            <tr>
                <td><strong>{name}</strong></td>
                <td>{r['accuracy']:.4f}</td>
                <td>{r['precision']:.4f}</td>
                <td>{r['recall']:.4f}</td>
                <td>{r['f1']:.4f}</td>
                <td>{r['auc']:.4f}</td>
                <td>${r.get('total_cost', 0):,.2f}</td>
            </tr>
        """

html_content += """
        </table>
    </div>

    <div class="section">
        <h2>📋 Confusion Matrices</h2>
"""

for name, r in state.results.items():
    html_content += f"""
        <h3>{name}</h3>
        <table style="width:300px;">
            <tr><td></td><td><strong>Pred +</strong></td><td><strong>Pred -</strong></td></tr>
            <tr><td><strong>Actual +</strong></td><td>{r['tp']}</td><td>{r['fn']}</td></tr>
            <tr><td><strong>Actual -</strong></td><td>{r['fp']}</td><td>{r['tn']}</td></tr>
        </table>
        <p>Value Breakdown: TP(${r['tp']*state.cost_tp:,.0f}) + FP(${r['fp']*state.cost_fp:,.0f}) + TN(${r['tn']*state.cost_tn:,.0f}) + FN(${r['fn']*state.cost_fn:,.0f}) = <strong>${r.get('total_cost',0):,.2f}</strong></p>
    """

html_content += """
    </div>
</body>
</html>
"""

# Save and download
filename = f"ML_Results_{datetime.now().strftime('%Y%m%d_%H%M')}.html"
with open(filename, 'w') as f:
    f.write(html_content)

print(f"✅ Report generated!")
files.download(filename)

ModuleNotFoundError: No module named 'google'

In [ ]:
#@title 📦 **Download Trained Models** { display-mode: "form" }
#@markdown Download your trained models as .pkl files to use later.

import joblib
import zipfile
from google.colab import files
from datetime import datetime

if len(state.models) == 0:
    print("❌ No models trained yet!")
else:
    # Select which model(s) to download
    print("📦 Available models:", list(state.models.keys()))

    download_all = True  #@param {type:"boolean"}

    if download_all:
        # Save all models + scaler in a zip file
        zip_name = f"ML_Models_{datetime.now().strftime('%Y%m%d_%H%M')}.zip"

        with zipfile.ZipFile(zip_name, 'w') as zipf:
            # Save each model
            for name, model in state.models.items():
                model_file = f"{name.replace(' ', '_')}.pkl"
                joblib.dump(model, model_file)
                zipf.write(model_file)
                print(f"   ✅ {name} saved")

            # Save scaler if exists
            if state.scaler is not None:
                joblib.dump(state.scaler, 'scaler.pkl')
                zipf.write('scaler.pkl')
                print(f"   ✅ Scaler saved")

            # Save feature names
            with open('feature_names.txt', 'w') as f:
                f.write('\n'.join(state.feature_names))
            zipf.write('feature_names.txt')
            print(f"   ✅ Feature names saved")

        print(f"\n📥 Downloading {zip_name}...")
        files.download(zip_name)

    print("""

📌 To load a model later:
   import joblib
   model = joblib.load('Ridge.pkl')
   scaler = joblib.load('scaler.pkl')

   # For new predictions:
   X_new_scaled = scaler.transform(X_new)  # For Ridge/Lasso
   predictions = model.predict(X_new_scaled)
    """)